# Atividade Prática 3

## Transformada de Fourier para imagens

### Entrega: até 24/04/2020 às 23:59 no e-disciplinas

#### Nome: ________________________ N° USP: ________  ( ) Grad ( ) Pós

---
###  

A transformada de Fourier de uma imagem $A\in\mathcal{M}_{m,n}(\mathbb{C})$ segue bem de perto o exemplo 1.24, e corresponde aos coeficientes não-normalizados da mudança de base da representação usual (base canônica) para a base das formas básicas de onda bidimensionais dadas pelas matrizes $\mathcal{E}_{k,l}$, definidas na seção 1.7.2 como

$$\left(\mathcal{E}_{k,l}\right)_{r,s} = e^{i2\pi(kr/m+ls/n)},\quad r=0,\ldots,m-1,\quad s=0,\ldots,n-1,$$

onde escrevemos $A=\displaystyle\sum_{k=0}^{m-1}\sum_{l=0}^{n-1}c_{k,l}\mathcal{E}_{k,l}$ e obtemos os coeficientes $c_{k,l}$ pelo teorema 1.8.3 como $c_{k,l}=\frac{\left(A,\mathcal{E}_{k,l}\right)}{mn}$.

Especificamente, definimos $\hat{A}=DFT(A)\in\mathcal{M}_{m,n}(\mathbb{C})$ como a matriz cujas componentes são

$$\hat{A}_{k,l} = \left(A,\mathcal{E}_{k,l}\right) = \sum_{r=0}^{m-1}\sum_{s=0}^{n-1}A_{r,s}e^{-i2\pi(kr/m+ls/n)},\quad k=0,\ldots,m-1,\quad l=0,\ldots,n-1.$$

Observe que nessa definição, $k$ e $l$ são índices associados às *frequências de varredura por linhas e por colunas*, respectivamente, enquanto $r$ e $s$ são os índices de linhas e colunas dentro das matrizes $A$ e $\mathcal{E}_{k,l}$.

---

**Exercício 1a:** Adapte o código da implementação "ingênua" da DFT unidimensional, fornecido abaixo, para produzir uma implementação análoga da DFT bidimensional.

In [88]:
import math as m
import numpy as np

def DFTingenua(x):
    N = len(x); n = np.array(range(N))
    X = np.ndarray(N, dtype=complex)
    for k in range(N):
        X[k] = sum(x*np.exp(-1j*2*m.pi*k*n/N))
    return X

In [89]:
# Resposta do exercício 1a
def DFT2ingenua(A):
    # begin gabarito
    # obs: nessa implementação, a matriz auxiliar E_{k,l} é construída
    # usando meshgrid, convertendo a expressão e^(i2π(kr/M+ls/N)) em uma matriz
    # (isso poderia ser feito com mais um laço duplo em (r,s), com a função np.inner,
    # ou ainda usando list comprehension como no gabarito do exercício 1b).
    (M,N) = np.shape(A); r,s = np.meshgrid(range(M),range(N),indexing='ij')
    Ahat = np.ndarray((M,N), dtype=complex)
    for k in range(M):
        for l in range(N):
            Ahat[k,l] = sum(sum(A*np.exp(-1j*2*m.pi*(k*r/M+l*s/N))))
    # end gabarito
    return Ahat

---

### Testes automatizados

É possível testar automaticamente o código da DFT comparando as saídas da função implementada com valores pré-calculados a partir da definição. Por exemplo, na DFT 1D vimos (exercício 2.4 da última lista) que os vetores $e_r=(0,\ldots,0,\overbrace{1}^{i=r},0,\ldots,0)$ da base canônica possuem como DFT os vetores
$$DFT(e_r) = \overline{E_r} = (e^0,e^{-i2\pi r/N},e^{-i2\pi 2r/N},\ldots,e^{-i2\pi (N-1)r/N}).$$

Esses casos de teste são utilizados na função abaixo. Para cada $e_r$ ela constrói a DFT teórica e a compara com o resultado da chamada da função DFTingenua. Observe que o código só imprime a frase no final porque todas as comparações (asserts) estão muito próximas do esperado (o código usa $\varepsilon=10^{-8}$ como tolerância nas comparações).

In [90]:
# teste automatizado:
# para cada vetor e_r da base canônica, computa sua DFT
# e compara com a expressão teórica DFT(e_r)=\overline{E_r} (E_r conjugado)
def testeDFTingenua(N):
    for r in range(N):
        e_r = np.zeros(N); e_r[r] = 1
        DFTe_r = [ np.exp(-1j*2*m.pi*r*k/N) for k in range(N) ]
        assert np.linalg.norm(DFTingenua(e_r)-DFTe_r) < 1e-8

testeDFTingenua(4)
print("A função DFTingenua passou no teste!")

A função DFTingenua passou no teste!


---

**Exercício 1b:** Adapte o código do teste automatizado da DFT unidimensional fornecido acima para testar sua implementação da DFT bidimensional. Seu teste deve percorrer todas as matrizes da base canônica de $\mathcal{M}_{m,n}(\mathbb{C})$ e comparar o resultado de sua função DFT2ingenua com a DFT prevista pela definição. Você deve construir essas DFTs previstas a partir das expressões teóricas: dada uma matriz da base canônica $e_{r,s}\in\mathcal{M}_{m,n}(\mathbb{C})$ definida por $(e_{r,s})_{i,j} = 1$ se $(r,s)=(i,j)$ e $(e_{r,s})_{i,j} = 0$ caso contrário, sua DFT será a matriz $\hat{e}_{r,s}\in\mathcal{M}_{m,n}(\mathbb{C})$ cujas componentes são

$$\left(\hat{e}_{r,s}\right)_{k,l} = e^{-i2\pi(kr/m+ls/n)},\quad k=0,\ldots,m-1,\quad l=0,\ldots,n-1.$$

In [91]:
# Resposta do exercício 1b
def testeDFT2ingenua(M,N):
    # begin gabarito
    # obs: nessa implementação, a matriz ê_{rs} é construída usando list comprehension
    # (poderia também ser feito com meshgrid como no exercício 1a, ou com um laço duplo).
    for r in range(M):
        for s in range(N):
            E_rs = np.zeros((M,N)); E_rs[r,s] = 1
            DFTE_rs = [ [ np.exp(-1j*2*m.pi*(r*k/M+s*l/N)) for l in range(N) ] for k in range(M) ]
            assert np.linalg.norm(DFT2ingenua(E_rs)-DFTE_rs) < 1e-8
    # end gabarito

testeDFT2ingenua(4,8)
print("A função DFT2ingenua passou no teste!")

A função DFT2ingenua passou no teste!


---

## DFT 2D como DFT nas linhas e nas colunas de A

Uma relação muito importante entre a DFT unidimensional e a DFT bidimensional é a seguinte: se $\hat{A} = DFT(A)$, onde $A,\hat{A}\in\mathcal{M}_{m,n}(\mathbb{C})$, então

$$\hat{A} = F_mAF_n^T,$$

onde $F_m\in\mathcal{M}_{m,m}(\mathbb{C})$ e $F_n\in\mathcal{M}_{n,n}(\mathbb{C})$ são as matrizes que representam as DFTs unidimensionais nos espaços $\mathbb{C}^m$ e $\mathbb{C}^n$, respectivamente.

Para ver que isso é verdade, basta calcular o produto de matrizes acima:

$$\begin{array}{ll}
\left(F_mAF_n^T\right)_{k,l} & = ((F_mA)F_n^T)_{k,l}\\
& = \mbox{"linha k de $F_mA$" $\quad\times\quad$ "coluna l de $F_n^T$"}\\
& = \mbox{"linha k de $F_mA$" $\quad\times\quad$ "linha l de $F_n$"}\\
& = \displaystyle\sum_{s=0}^{n-1}(FmA)_{k,s}e^{-i2\pi ls/n}\\
& = \displaystyle\sum_{s=0}^{n-1}\left(\sum_{r=0}^{m-1}A_{r,s}e^{-i2\pi kr/m}\right)e^{-i2\pi ls/n}\\
& = \displaystyle\sum_{r=0}^{m-1}\sum_{s=0}^{n-1}A_{r,s}e^{-i2\pi(kr/m+ls/n)}\\
& = \hat{A}_{k,l}
\end{array}$$

A interpretação algorítmica da identidade $DFT(A) = F_mAF_n^T$ é a seguinte:

- A parte $F_mA$ pode ser vista como uma aplicação da DFT nas **colunas** de A:

$$F_mA = \left[\begin{array}{l|l|l|l}
\rule{0mm}{0mm}\\
F_mA^0&F_mA^1&\ldots&F_mA^{n-1}\\
\rule{0mm}{0mm}
\end{array}\right]
 = \left[\begin{array}{l|l|l|l}
\rule{0mm}{0mm}\\
DFT(A^0)&DFT(A^1)&\ldots&DFT(A^{n-1})\\
\rule{0mm}{0mm}
\end{array}\right]$$

- O produto subsequente por $F_n^T$ pode ser visto como uma aplicação da DFT nas **linhas** da matriz resultante $X=F_mA$, pois

$$XF_n^T = \left[\begin{array}{c}
\rule{2cm}{0mm}X_0F_n^T\rule{2cm}{0mm}\\\hline
X_1F_n^T\\\hline
\vdots\\\hline
X_{m-1}F_n^T
\end{array}\right]
 = \left[\begin{array}{c}
\rule{2cm}{0mm}DFT(X_0)^T\rule{2cm}{0mm}\\\hline
DFT(X_1)^T\\\hline
\vdots\\\hline
DFT(X_{m-1})^T
\end{array}\right]$$

- Finalmente, a associatividade do produto de matrizes $$F_mAF_n^T=(F_mA)F_n^T=F_m(AF_n^T)$$ permite ver que o mesmo resultado poderia ser atingido calculando-se primeiro a DFT das linhas de $A$ e depois a DFT das colunas resultantes.

---

**Exercício 2a:** Use a implementação da FFT unidimensional abaixo para construir uma implementação da FFT bidimensional usando a identidade $DFT(A) = F_mAF_n^T$.

In [92]:
def FFT(x):
    N = len(x);
    if N<=1: return x
    X = np.ndarray(N, dtype=complex)
    Xpar = FFT(x[0:N:2])
    Ximpar = FFT(x[1:N:2])
    for k in range(N):
        X[k] = Xpar[k%(N//2)]+np.exp(-1j*2*m.pi*k/N)*Ximpar[k%(N//2)]
    return X

In [93]:
# Resposta do exercício 2a
def FFT2(A):
    # begin gabarito
    (M,N) = np.shape(A);
    Ahat = np.ndarray((M,N), dtype=complex)
    for r in range(M):
        Ahat[r,:] = FFT(A[r,:])
    for s in range(N):
        Ahat[:,s] = FFT(Ahat[:,s])
    # end gabarito
    return Ahat

---

**Exercício 2b:** Escreva uma função para testar sua implementação da FFT bidimensional, comparando os resultados dessa função com a sua implementação DFT2ingenua.

In [94]:
# Resposta do exercício 2b
def testeFFT2(M,N):
    # begin gabarito
    for r in range(M):
        for s in range(N):
            E_rs = np.zeros((M,N)); E_rs[r,s] = 1
            assert np.linalg.norm(DFT2ingenua(E_rs)-FFT2(E_rs)) < 1e-8
    # end gabarito

testeFFT2(4,8)
print("A função FFT2 passou no teste!")

A função FFT2 passou no teste!
